In [83]:
#imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import pandas as pd

In [84]:
#set up api key variables
keysfile = open('apikeys','r')
cl_id = keysfile.readline().strip()
cl_sec = keysfile.readline().strip()

#set up spotify object
auth_manager = SpotifyClientCredentials(client_id=cl_id, client_secret=cl_sec)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [85]:
#requests to get all of moby's albums released in the US (which removes a bunch of duplicates)
moby_uri = 'spotify:artist:3OsRAKCvk37zwYcnzRf5XF'

results = sp.artist_albums(moby_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

albums = [album for album in albums if 'US' in album['available_markets'] ]

# for album in albums:
#     print(album['name'])


In [86]:
#get all the track ids and names from all of the albums
album_ids = [album['id'] for album in albums]
album_ids

track_ids = []
track_names = []
for ai in album_ids:
    results = sp.album_tracks(ai)
    tracks = results['items']
    track_ids.extend([track['id'] for track in tracks])
    track_names.extend([track['name'] for track in tracks])

In [87]:
#grab the audio features for all the tracks in batches of 100
data = sp.audio_features(track_ids[0:100])
data.extend(sp.audio_features(track_ids[100:200]))
data.extend(sp.audio_features(track_ids[200:300]))
data.extend(sp.audio_features(track_ids[300:400]))
data.extend(sp.audio_features(track_ids[400:500]))
data.extend(sp.audio_features(track_ids[500:600]))
data.extend(sp.audio_features(track_ids[600:692]))

In [88]:
#put it in a dataframe with names, drop unnecessary fields, export
df = pd.DataFrame(data)
df.insert(0,'name',track_names,True)
df.drop(['type','uri','track_href','analysis_url'], axis=1, inplace=True)
df.to_csv('moby_song_data.txt')